# 原始矩阵正向化函数

In [1]:
import numpy as np
# 极小型指标转化为极大型指标的函数
def minTomax(maxx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    ans = [[(maxx-e)] for e in x]  # 计算最大值与每个指标值的差，并将其放入新列表中
    return np.array(ans)  # 将列表转换为numpy数组并返回

# 中间型指标转化为极大型指标的函数
def midTomax(bestx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    h = [abs(e-bestx) for e in x]  # 计算每个指标值与最优值之间的绝对差
    M = max(h)  # 找到最大的差值
    if M == 0:
        M = 1  # 防止最大差值为0的情况
    ans = [[(1-e/M)] for e in h]  # 计算每个差值占最大差值的比例，并从1中减去，得到新指标值
    return np.array(ans)  # 返回处理后的numpy数组

# 区间型指标转化为极大型指标的函数
def regTomax(lowx, highx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    M = max(lowx-min(x), max(x)-highx)  # 计算指标值超出区间的最大距离
    if M == 0:
        M = 1  # 防止最大距离为0的情况
    ans = []
    for i in range(len(x)):
        if x[i]<lowx:
            ans.append([(1-(lowx-x[i])/M)])  # 如果指标值小于下限，则计算其与下限的距离比例
        elif x[i]>highx:
            ans.append([(1-(x[i]-highx)/M)])  # 如果指标值大于上限，则计算其与上限的距离比例
        else:
            ans.append([1])  # 如果指标值在区间内，则直接取为1
    return np.array(ans)  # 返回处理后的numpy数组


# 数据初始化

In [2]:

A = [[9,10,175,120], [8,7,164,80],[6,3,157,90]]
A = np.array(A)
objectNum = len(A)
indicatorNum = len(A[0])

# 指标类型，按顺序输入，1:极大型，2：极小型，3：中间型，4：区间型
kind = [1,2,3,4]

# 中间值的最优值
bestA = 165
# 区间型的最大值和最小值
lowA = 90
highA = 100

# Step1. 矩阵正向化

In [3]:
X = np.zeros(shape=(objectNum, 0))  # 初始化为 0 列，因为后续会逐列拼接

for i in range(indicatorNum):
    if kind[i] == 1:  # 如果当前指标为极大型，则直接使用原值
        v = np.array(A[:, i])
    elif kind[i] == 2:  # 如果当前指标为极小型，调用 minTomax 函数转换
        maxA = max(A[:, i])
        v = minTomax(maxA, A[:, i])
    elif kind[i] == 3:  # 如果当前指标为中间型，调用 midTomax 函数转换
        v = midTomax(bestA, A[:, i])
    elif kind[i] == 4:  # 如果当前指标为区间型，调用 regTomax 函数转换
        v = regTomax(lowA, highA, A[:, i])
    
    if i==0:
        X = v.reshape(-1, 1)
    else:
        X = np.hstack([X, v])  # 否则，将新指标列拼接到 X 数组上

print("统一指标后矩阵为：\n{}".format(X))  # 打印处理后的矩阵 X

统一指标后矩阵为：
[[9.  0.  0.  0. ]
 [8.  3.  0.9 0.5]
 [6.  7.  0.2 1. ]]


# Step2. 标准化正向矩阵

In [4]:
X = X.astype('float')  # 确保X矩阵的数据类型为浮点数
for j in range(indicatorNum):
    col_min = np.min(X[:, j])
    col_max = np.max(X[:, j])
    
    if col_min < 0:  # 如果该列存在负数，使用 [0, 1] 区间的标准化公式
        print("col_min < 0,包含负数")
        X[:, j] = (X[:, j] - col_min) / (col_max - col_min)
    else:
        # 原标准化方法
        X[:, j] = X[:, j] / np.sqrt(sum(X[:, j]**2))
print("标准化矩阵为：\n{}".format(X))  # 打印标准化后的矩阵X

标准化矩阵为：
[[0.66896473 0.         0.         0.        ]
 [0.59463532 0.3939193  0.97618706 0.4472136 ]
 [0.44597649 0.91914503 0.21693046 0.89442719]]


# Step3. 得到权重值
使用：
- 熵权法
- AHP
····

# Step4. 计算得分

In [5]:
# 最大值最小值距离的计算
x_max = np.max(X, axis=0)  # 计算标准化矩阵每列的最大值
x_min = np.min(X, axis=0)  # 计算标准化矩阵每列的最小值
d_z = np.sqrt(np.sum(np.square((X - np.tile(x_max, (objectNum, 1)))), axis=1))  # 计算每个参评对象与最优情况的距离d+
d_f = np.sqrt(np.sum(np.square((X - np.tile(x_min, (objectNum, 1)))), axis=1))  # 计算每个参评对象与最劣情况的距离d-
print('每个指标的最大值:', x_max)
print('每个指标的最小值:', x_min)
print('d+向量:', d_z)
print('d-向量:', d_f)

# 计算每个参评对象的得分排名
s = d_f/(d_z+d_f)  # 根据d+和d-计算得分s，其中s接近于1则表示较优，接近于0则表示较劣
Score = 100*s/sum(s)  # 将得分s转换为百分制，便于比较
for i in range(len(Score)):
    print(f"第{i+1}个标准化后百分制得分为：{Score[i]}")  # 打印每个参评对象的得分

每个指标的最大值: [0.66896473 0.91914503 0.97618706 0.89442719]
每个指标的最小值: [0.44597649 0.         0.         0.        ]
d+向量: [1.61175952 0.69382053 0.79132442]
d-向量: [0.22298824 1.15334862 1.30072534]
第1个标准化后百分制得分为：8.886366735657832
第2个标准化后百分制得分为：45.653341055701134
第3个标准化后百分制得分为：45.46029220864103
